# Chemanim Video Rendering (GPU)
Render chemistry and protein animations with GPU acceleration using [chemanim](https://pypi.org/project/chemanim/).

## Step 1: Install System Dependencies

In [ ]:
# System dependencies for OpenGL rendering
!apt-get update -qq
!apt-get install -y -qq xvfb libgl1-mesa-dev libglu1-mesa-dev freeglut3-dev mesa-utils libosmesa6-dev libglfw3 libglfw3-dev libcairo2-dev libpango1.0-dev ffmpeg texlive-latex-base

## Step 2: Install Python Packages

In [ ]:
# Install manimgl first (has complex dependencies)
!pip install manimgl

# Install chemanim with bio dependencies
!pip install chemanim[bio]

print('\n✓ Installation complete!')

In [ ]:
# Start virtual display for headless rendering
import subprocess
import os

# Kill any existing Xvfb
!pkill -9 Xvfb 2>/dev/null || true

# Start fresh Xvfb
subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1920x1080x24'])
os.environ['DISPLAY'] = ':99'

# Wait for it to start
import time
time.sleep(2)
print('✓ Virtual display started')

In [ ]:
# Create ManimGL config for 60fps HD rendering
%%writefile custom_config.yml
directories:
  output: ./output

camera:
  fps: 60
  pixel_width: 1920
  pixel_height: 1080

In [ ]:
# Verify installation
from chemanim.bio import Protein
from chemanim.chem_object import ChemObject
print('✓ Chemanim imported successfully!')

## Step 3: Create Your Animation
Edit the cell below to customize your animation.

In [ ]:
%%writefile my_animation.py
from manimlib import *
from chemanim.bio import Protein

class ProteinShowcase(Scene):
    def construct(self):
        # Setup 3D camera
        frame = self.camera.frame
        frame.set_euler_angles(theta=30 * DEGREES, phi=70 * DEGREES)
        frame.scale(3.0)
        
        # Title
        title = Text("Crambin Protein (1CRN)", font_size=40).to_edge(UP)
        title.fix_in_frame()
        self.add(title)
        
        # Fetch and display protein
        mol = Protein.from_pdb_id("1CRN", download_dir=".", include_hydrogens=False)
        mol.apply_render_style("cartoon")
        mol.set_color_scheme("sse")  # Color by secondary structure
        mol.scale(2.0)
        mol.move_to(ORIGIN)
        
        self.play(FadeIn(mol), run_time=1.5)
        
        # Rotate and show different color schemes
        for scheme in ["chain", "rainbow", "amino_acid"]:
            mol.set_color_scheme(scheme)
            label = Text(f"Color: {scheme}", font_size=24).to_edge(DOWN)
            label.fix_in_frame()
            self.add(label)
            self.play(Rotate(mol, angle=90*DEGREES, axis=UP), run_time=1.5)
            self.remove(label)
        
        self.wait(0.5)

## Step 4: Render Video

In [ ]:
# Create output directory
!mkdir -p ./output

# Render the animation using config file (avoids --fps bug in ManimGL v1.7.2)
!python -m manimlib my_animation.py ProteinShowcase -w --hd --config_file custom_config.yml

## Step 5: Find and Download Video

In [ ]:
# Debug: Show what files exist
print('=== Output directory ===')
!ls -la ./output/ 2>/dev/null || echo 'output/ not found'
print('\n=== Current directory ===')
!ls -la *.mp4 2>/dev/null || echo 'No mp4 in current dir'
print('\n=== All .mp4 files (searching everywhere) ===')
!find /content -name '*.mp4' 2>/dev/null
!find ~ -name '*.mp4' 2>/dev/null

In [ ]:
import glob
import os
from pathlib import Path
from google.colab import files

# Search everywhere for mp4 files
home = str(Path.home())
search_patterns = [
    '*.mp4',
    'output/*.mp4',
    'output/**/*.mp4',
    f'{home}/videos/**/*.mp4',
    f'{home}/**/*.mp4',
    '/content/**/*.mp4',
]

videos = []
for pattern in search_patterns:
    try:
        found = glob.glob(pattern, recursive=True)
        videos.extend(found)
    except:
        pass

# Remove duplicates
videos = list(set(videos))
print(f'Found {len(videos)} video(s):')
for v in videos:
    size = os.path.getsize(v) / (1024*1024)
    print(f'  - {v} ({size:.2f} MB)')

if videos:
    print('\nDownloading...')
    for v in videos:
        files.download(v)
else:
    print('\n⚠️ No videos found!')
    print('Check Step 4 output for errors.')

---
## Alternative: Simple Test Animation
If the protein animation has errors, try this simple test first:

In [ ]:
%%writefile simple_test.py
from manimlib import *

class SimpleTest(Scene):
    def construct(self):
        # Just a simple circle animation
        circle = Circle(color=BLUE, fill_opacity=0.5)
        text = Text("ManimGL Works!", font_size=48).to_edge(UP)
        
        self.play(Write(text))
        self.play(Create(circle))
        self.play(circle.animate.scale(2).set_color(RED))
        self.play(Rotate(circle, angle=TAU))
        self.wait(1)

In [ ]:
# Render simple test (without --fps to avoid the bug)
!python -m manimlib simple_test.py SimpleTest -w --hd --config_file custom_config.yml

---
## Example: Molecule Animation

In [ ]:
%%writefile molecule_demo.py
from manimlib import *
from chemanim.chem_object import ChemObject

class MoleculeDemo(Scene):
    def construct(self):
        # Setup 3D camera
        frame = self.camera.frame
        frame.set_euler_angles(theta=20 * DEGREES, phi=75 * DEGREES)
        frame.scale(2.0)
        
        # Create molecules
        water = ChemObject.from_compound_name("water")
        caffeine = ChemObject.from_compound_name("caffeine")
        
        water.render_style = "ball_and_stick"
        caffeine.render_style = "ball_and_stick"
        
        water.scale(1.5).shift(LEFT * 3)
        caffeine.scale(1.5).shift(RIGHT * 2)
        
        # Labels
        water_label = Text("H2O", font_size=30).next_to(water, DOWN)
        water_label.fix_in_frame()
        caff_label = Text("Caffeine", font_size=30).next_to(caffeine, DOWN)
        caff_label.fix_in_frame()
        
        self.play(FadeIn(water), FadeIn(water_label))
        self.play(FadeIn(caffeine), FadeIn(caff_label))
        self.play(
            Rotate(water, angle=TAU, axis=UP),
            Rotate(caffeine, angle=TAU, axis=UP),
            run_time=3
        )
        self.wait(1)

In [ ]:
# Render molecule demo
!python -m manimlib molecule_demo.py MoleculeDemo -w --hd --config_file custom_config.yml